In [49]:
import pandas as pd

In [50]:
maple_train_df = pd.read_csv('maple_train.csv')
maple_test_df = pd.read_csv('maple_test.csv')
maple_eval_df = pd.read_csv('maple_eval.csv')
#

In [51]:
# maple_train_df[maple_train_df["Path"] == "nitrogen"]

In [52]:
import os
path = './PathwayFeatures'
pathway_names = []
pw_names = maple_eval_df["Path"].unique()
for i, attrlist in enumerate(os.listdir(path)):
    attributes = pd.read_csv(path+'/'+ attrlist)
    attributes = attributes["Attrib"].tolist()
    pathway_name = attrlist.split('-')[0]
    pathway_names.append(pathway_name)

    maple_train_df.loc[maple_train_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_test_df.loc[maple_test_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'
    maple_eval_df.loc[maple_eval_df['Path'] == pathway_name.replace('_', ' '), 'Class'] = 'positive'

    maple_train_df.loc[maple_train_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_test_df.loc[maple_test_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'
    maple_eval_df.loc[maple_eval_df['Path'] != pathway_name.replace('_', ' '), 'Class'] = 'negative'

    train_df_prep = maple_train_df[attributes]
    test_df_prep = maple_test_df[attributes]
    eval_df_prep = maple_eval_df[attributes]

    train_df_prep.to_csv('Datasets/'+pathway_name+'_train.csv', index=False)
    test_df_prep.to_csv('Datasets/'+pathway_name+'_test.csv', index=False)
    eval_df_prep.to_csv('Datasets/'+pathway_name+'_eval.csv', index=False)


In [123]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
# by: @invoktheshell
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)):
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, recall_score, precision_score

def calc_metrics(y_true, y_pred, model_name, pw_name, dataset_name, fold_num):
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    tp, fp, tn, fn = perf_measure(y_true, y_pred)
    recall = precision_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    specifity = tn / (tn + fp)
    # print(precision, specifity, tp,fp,tn,fn)
    try:
        fpr = fp / (fp + tn)
    except:
        fpr = 0
    try:
        tpr = tp / (tp + fn)
    except:
        tpr = 0
    try:
        fnr = fn / (fn + tp)
    except:
        fnr = 0
    try:
        fdr = fp / (fp + tp)
    except:
        fdr = 0
    return model_name, pw_name, dataset_name, fold_num, acc, f1, mcc, tp, fp, tn, fn, recall, precision, specifity, fpr, tpr, fnr, fdr

In [124]:
#.3 Metabolic pathway modeling and analysis
# The 10-fold cross-validation and bagging were used on all training process,
# and the metrics accuracy, percentage of correctly classified instances (CCI),
# true positive (TP) rate, false positive (FP) rate, false negative (FN) rate, recall,
# specificity, F-measure, false discovery rate (FDR), and Matthews coefficient correlation (MCC)
# were used to evaluate the predictive performances.

import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import SGDClassifier
# from collections import defaultdict
# res_dict = defaultdict(list)
res = []
for pathway_name in pathway_names:
    # if pathway_name=='amine_and_polyamine':
    train_df = pd.read_csv('Datasets/'+ pathway_name+'_train.csv')
    # test_df = pd.read_csv('Datasets/'+ pathway_name+'_test.csv')
    # eval_df = pd.read_csv('Datasets/'+ pathway_name+'_eval.csv')
    print('pathway: ', pathway_name)
    # print(len(train_df[train_df["Class"]=='positive']))
    train_df["Class"] = train_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # test_df["Class"] = test_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    # eval_df["Class"] = eval_df["Class"].apply(lambda x: 1 if x=='positive' else 0)
    names = ['svc', 'nb', 'dt', 'mlp', 'knn', 'rf', 'sgd']
    models = [BaggingClassifier(SVC()),
              BaggingClassifier(GaussianNB()),
              BaggingClassifier(DecisionTreeClassifier()),
              BaggingClassifier(MLPClassifier()),
              BaggingClassifier(KNeighborsClassifier()),
              BaggingClassifier(RandomForestClassifier()),
              BaggingClassifier(SGDClassifier()),]
    skf = StratifiedKFold(n_splits=10, random_state=1337)
    X = train_df.drop('Class', axis=1)
    y = train_df["Class"]
    for i in range(len(models)):
        fold_num=0
        for train_index, test_index in skf.split(X, y):
            fold_num+=1
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            models[i].fit(X_train, y_train)
            preds = models[i].predict(X_test)
            # print(y_test[y_test[]])
            res.append(calc_metrics(y_test.tolist(), preds.tolist(), names[i], pathway_name, 'test_df', fold_num))

res_df = pd.DataFrame(res, columns=["model_name", "pw_name", "dataset_name", "fold_num", "acc", "f1", "mcc", "tp", "fp", "tn", "fn", "recall", "precision", "specifity", "fpr", "tpr", "fnr", "fdr"])
res_df.to_csv('10fold_results.csv', index=False)
del y_test, y_train, X_test, X_train, X, y

pathway:  amine_and_polyamine


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


IndexError: list index out of range

In [135]:
pd.read_csv('10fold_results.csv').drop('fold_num', axis=1).groupby('model_name').mean()

,acc,f1,mcc,tp,fp,tn,fn,recall,precision,specifity,fpr,tpr,fnr,fdr
model_name,,,,,,,,,,,,,,
dt,0.986336,0.799392,0.799200,73.5,17.0,2633.5,20.5,0.842650,0.842650,0.993585,0.006415,0.781915,0.218085,0.157350
knn,0.994899,0.917251,0.919023,80.0,0.0,2650.5,14.0,1.000000,1.000000,1.000000,0.000000,0.851064,0.148936,0.000000
mlp,0.987064,0.826506,0.829699,80.5,22.0,2628.5,13.5,0.833333,0.833333,0.991698,0.008302,0.856383,0.143617,0.166667
nb,0.771891,0.216178,0.268064,79.0,611.0,2039.5,15.0,0.126741,0.126741,0.769459,0.230541,0.840426,0.159574,0.873259
rf,0.989980,0.827739,0.836314,66.5,0.0,2650.5,27.5,1.000000,1.000000,1.000000,0.000000,0.707447,0.292553,0.000000
svc,0.995082,0.920074,0.921853,80.5,0.0,2650.5,13.5,1.000000,1.000000,1.000000,0.000000,0.856383,0.143617,0.000000


In [126]:
# The best classifiers for each metabolic pathway were ranked using the Kruskal-Wallis test.
# Bu test algoritmaları nasıl sıralıyor anlamadım, benim anladığım fark varmı diye bakıyor dağılımlarda

from scipy.stats import kruskal
res_rows = res_df.drop('fold_num', axis=1).groupby('model_name').mean()
for i in range(len(res_rows.index)):
    for j in range(i+1, len(res_rows.index)):
        kr = kruskal(res_rows.loc[str(res_rows.index[i])], res_rows.loc[str(res_rows.index[j])])
        print(kr)
        if kr.pvalue < 0.05:
            print("statistically significant p value is found: ", str(res_rows.index[i], str(res_rows.index[j])))

KruskalResult(statistic=0.03386243386243255, pvalue=0.8539997400757506)
KruskalResult(statistic=0.03379752777343008, pvalue=0.8541381599961391)
KruskalResult(statistic=0.6104678454076083, pvalue=0.43461171245060526)
KruskalResult(statistic=0.01904761904762454, pvalue=0.8902300549434817)
KruskalResult(statistic=0.03386243386243255, pvalue=0.8539997400757506)
KruskalResult(statistic=0.03386243386243255, pvalue=0.8539997400757506)
KruskalResult(statistic=0.1714285714285639, pvalue=0.6788452994243248)
KruskalResult(statistic=0.043598740082131965, pvalue=0.8346019409234209)
KruskalResult(statistic=0.013456401259915306, pvalue=0.9076511710638686)
KruskalResult(statistic=0.4752777343138738, pvalue=0.4905691312298255)
KruskalResult(statistic=0.03386243386243255, pvalue=0.8539997400757506)
KruskalResult(statistic=0.04289244528124076, pvalue=0.8359279247111269)
KruskalResult(statistic=0.07619047619046967, pvalue=0.7825279247400765)
KruskalResult(statistic=0.1714285714285639, pvalue=0.67884529942

In [132]:
# her bir pathway için en iyi 3 classifier'ı seç (fp ve f measuer + kruskal-wallis test)
# Seçilen classifierları votingclassifier ile birleştirip parametre optimizasyonu yap
# burayı otomatikleştirmedim hangi pathwayler için hangi algoritmaların seçileceği belli olmadığı için

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
# örnek votingclassifier modeli amine_and_polyamine pathwayi için
amine_and_polyamine_model = VotingClassifier(estimators=[
                        ('svc', BaggingClassifier(SVC())),
                        ('nb', BaggingClassifier(GaussianNB())),
                        ('dt', BaggingClassifier(DecisionTreeClassifier()))], voting='soft')
amino_acid_model = []
aromatic_compound_model = []
carbohydrate_model = []
carotenoid_model = []
energy_model = []
lipid_model = []
nitrogen_model = []
nucleotide_sugar_model = []
phospholipid_model = []
protein_modification_model = []
secondary_metabolite_model = []
models = [amine_and_polyamine_model]
parameters = []
# örnek parametre dictionary'si
parameters_amine_and_polyamine = {
                    'svc__base_estimator__C': [0.1, 1],
                    'svc__base_estimator__gamma': [1, 0.1],
                    'svc__base_estimator__kernel': ['rbf'],
                    'dt__base_estimator__max_depth': list(range(3,6,1)),
                    'dt__base_estimator__min_samples_leaf': list(range(4,10,2))
                }


parameters_rf = {
                'base_estimator__bootstrap': [True],
                'base_estimator__max_depth': [80, 90, 100],
                'base_estimator__max_features': [2, 3],
                'base_estimator__min_samples_leaf': [3, 4],
                'base_estimator__min_samples_split': [8, 10],
                'base_estimator__n_estimators': [100, 200]
                }
parameters_SVC={'base_estimator__C': [0.1, 1, 10, 100, 1000],
              'base_estimator__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'base_estimator__kernel': ['rbf']
                   }
parameters_MLP = {
    'base_estimator__hidden_layer_sizes': [(10,30,10),(20,)],
    'base_estimator__activation': ['tanh', 'relu'],
    'base_estimator__solver': ['sgd', 'adam'],
    'base_estimator__alpha': [0.0001, 0.05],
    'base_estimator__learning_rate': ['constant','adaptive'],
}
parameters_DT = {
    'base_estimator__max_depth': list(range(1,10,1)),
    'base_estimator__min_samples_leaf': list(range(1,10,2))
}
parameters_KNN = {
    'base_estimator__n_neighbors': list(range(1, 31))
}
parameters_SGD = {
    'base_estimator__C': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0],
    'base_estimator__penalty': ['l2'],
    'base_estimator__n_jobs': [-1]
}
parameters.append(parameters_amine_and_polyamine)
#her bir pathway için seçilen modeller o pathwayin train setinde parametre optimizasyonu yapılacak
for pathway_name in pathway_names:
    if pathway_name == 'amine_and_polyamine':
        df = pd.read_csv('Datasets/' + pathway_name+'_train.csv')
        for i, model in enumerate(models):
            clf = GridSearchCV(model, parameters, n_jobs=-1, verbose=3, scoring='f1_macro')
            clf.fit(df.drop('Class', axis=1), df["Class"])
            print(clf.best_score_)
            print(clf.best_params_)



Fitting 5 folds for each of 36 candidates, totalling 180 fits
0.9952639505324175
{'dt__base_estimator__max_depth': 5, 'dt__base_estimator__min_samples_leaf': 6, 'svc__base_estimator__C': 1, 'svc__base_estimator__gamma': 0.1, 'svc__base_estimator__kernel': 'rbf'}


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  9.0min finished


In [133]:

clf

GridSearchCV(estimator=VotingClassifier(estimators=[('svc',
                                                     BaggingClassifier(base_estimator=SVC())),
                                                    ('nb',
                                                     BaggingClassifier(base_estimator=GaussianNB())),
                                                    ('dt',
                                                     BaggingClassifier(base_estimator=DecisionTreeClassifier()))],
                                        voting='soft'),
             n_jobs=-1,
             param_grid=[{'dt__base_estimator__max_depth': [3, 4, 5],
                          'dt__base_estimator__min_samples_leaf': [4, 6, 8],
                          'svc__base_estimator__C': [0.1, 1],
                          'svc__base_estimator__gamma': [1, 0.1],
                          'svc__base_estimator__kernel': ['rbf']}],
             verbose=3)